# Разпознаване на навигация

Изграждайки си модел на HTML елементи да се провери дали използвайки линейна регресия можем да разпознаваме различни елементи от даден сайт, в частност основната навигация. 
Модела се базира на това елемента с какъв таг е дали е div nav или друг; дали в класовете на елемента присъстват nav, main, navigation, menu и прочие; колко линка се съдържат в зададеният елемент и колко от тях са на еднакаква дълбочина в DOM дървото.
    
Това което трябва да се добави е дълбочината на текущият елемент. И да се оптимизира да чете dataset-a от CSV

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mglearn
from IPython.display import display

%matplotlib notebook

import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")
warnings.filterwarnings(action="ignore", module="sklearn", message="^Objective did not")

## Прочитане на данните

Данните са генерирани на базата на следните критерии:

- tag
 - nav - 3
 - div - 2
 - header - 1
 - footer - -1
 - other - 0
- class
 - 2 - ако съдържа ключови думи за навигация
 - 1 - ако съдържа общи думи като main, menu
 - -1 - ако съдържа ключови думи за footer
- id - същото като за класа
- role
 - много висок приоритет за navigation
 - по-нисък приоритет за menu menubar & toolbar
 - отрицателен резултат за contentinfo, което би трябвало да е footer
- depth - на каква дълбочина е елемента в DOM дървото
- links - колко линка се съдържат в елемента
- links_by_depth - колко елемента са на една и съща дълбочина

В данните се съдържа колона probability която по горните критерии и тежести са зададени дали даден елемент е навигация или не.

In [2]:
df = pd.read_csv("linear-regression-input.csv", header=0)
df = df.drop_duplicates(subset=['url', 'tag', 'class', 'id', 'role', 'depth', 'links', 'links_by_depth'], keep='first')

Y = df['probability']
X = df[['tag', 'class', 'id', 'role', 'depth', 'links', 'links_by_depth']]

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=0)  

model = LinearRegression()

model.fit(X_train, y_train) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

## Коефициенти

Нека разгледаме какви са коефициентите на различните features. С какви 'тежести' участват.

In [4]:
coeff_df = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])  
coeff_df

,Coefficient
tag,0.039087
class,0.192208
id,0.075821
role,0.221869
depth,-0.002872
links,0.088649
links_by_depth,0.186838


Успеваемостта на текущият модел е:

In [5]:
model.score(X, Y)

0.9191943638537284

In [6]:
y_pred = model.predict(X_test) 
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})  
df

,Actual,Predicted
2159,0.40,0.433052
4780,0.40,0.453153
2954,0.40,0.458896
4254,0.40,0.258074
5546,0.25,0.260572
3007,0.40,0.450281
4852,0.35,0.364131
3118,0.40,0.447410
3771,0.15,0.183409
2114,0.40,0.441667


In [7]:
from sklearn import metrics  
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 

Mean Absolute Error: 0.053679156832543436
Mean Squared Error: 0.006121524122851664
Root Mean Squared Error: 0.07824016949656784
